In [11]:

install.packages('regsel',repos='http://cran.us.r-project.org')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


## Setup

### Importació de les dades.

In [2]:
library(ElemStatLearn)
data(spam)
wine<-read.csv('wine.csv',header=FALSE)
colnames(wine)<-c("Type","Alcohol","Malic","Ash","Alcalinity","Magnesium","Phenols","Flavonoids","Nonflavonoids","Proanthocyanins","Color","Hue","Dilution","Proline")
wine$Type <- as.factor(wine$Type)

### Llibreries

In [3]:
library(class)

## Classificació de Spam

Descripció de les dades **TODO**

In [4]:
nrow(spam)

[1] 4601

Veiem que el dataset és bastant gran aixi que utilitzarem com a model de predicció **k-fold cross-validation**. 
No utilitcem Leave-one-out per que seria poc eficient fer les 4601 fileres, tampoc utilitzarem boostrap ja que no tenim necessitat de ampliar la mostra.

Utilitzarem un k de 10

In [17]:
#Desordenem el dataset
spam<-spam[sample(nrow(spam)),]

### Classificació per  K-nn
Utilitzarem la funció *knn* que trobem al paquet *class*.
Amb la quantitat de dades que hi han testejarem amb una k= 10 i 20.

#### K= 10

In [6]:
k<-10

kfold<-10
totalprecisio.email<-0
totalprecisio.spam<-0
for (i in 1:kfold){
    ITest<-((nrow(spam)/kfold)*(i-1)+1):((nrow(spam)/kfold)*i)
    spam.train<-spam[-ITest,-ncol(spam)]
    spam.test<-spam[ITest,-ncol(spam)]
    spam.Ytrain<-spam[-ITest,ncol(spam)]
    spam.Ytest<-spam[ITest,ncol(spam)]
    spam.Yhat<-knn(spam.train, spam.test, spam.Ytrain, k = k)
    C<-table("True"=spam.Ytest,"Predicted"=spam.Yhat)
    cat('\nMatriu confusio\n')
    print(C)
    precisio.email<-C[1]/(C[1]+C[3])
    precisio.spam<-C[4]/(C[2]+C[4])
    cat('preciso email ',+precisio.email)
    cat('\nprecisio spam ',precisio.spam)
    cat('\n')
    totalprecisio.email<-totalprecisio.email + precisio.email
    totalprecisio.spam<-totalprecisio.spam + precisio.spam
}
cat('\npreciso email mitjana ',totalprecisio.email/kfold)
cat('\nprecisio spam mitjana ',totalprecisio.spam/kfold)


Matriu confusio
       Predicted
True    email spam
  email   231   58
  spam     45  126
preciso email  0.799308
precisio spam  0.7368421

Matriu confusio
       Predicted
True    email spam
  email   221   48
  spam     58  133
preciso email  0.8215613
precisio spam  0.6963351

Matriu confusio
       Predicted
True    email spam
  email   213   49
  spam     59  139
preciso email  0.8129771
precisio spam  0.7020202

Matriu confusio
       Predicted
True    email spam
  email   236   44
  spam     42  138
preciso email  0.8428571
precisio spam  0.7666667

Matriu confusio
       Predicted
True    email spam
  email   248   49
  spam     43  120
preciso email  0.8350168
precisio spam  0.7361963

Matriu confusio
       Predicted
True    email spam
  email   233   38
  spam     55  134
preciso email  0.8597786
precisio spam  0.7089947

Matriu confusio
       Predicted
True    email spam
  email   249   24
  spam     47  140
preciso email  0.9120879
precisio spam  0.7486631

Matriu confus

#### K= 20

In [69]:
k<-20

kfold<-10
totalprecisio.email<-0
totalprecisio.spam<-0
for (i in 1:kfold){
    ITest<-((nrow(spam)/kfold)*(i-1)+1):((nrow(spam)/kfold)*i)
    spam.train<-spam[-ITest,-ncol(spam)]
    spam.test<-spam[ITest,-ncol(spam)]
    spam.Ytrain<-spam[-ITest,ncol(spam)]
    spam.Ytest<-spam[ITest,ncol(spam)]
    spam.Yhat<-knn(spam.train, spam.test, spam.Ytrain, k = k)
    C<-table("True"=spam.Ytest,"Predicted"=spam.Yhat)
    cat('\nMatriu confusio\n')
    print(C)
    precisio.email<-C[1]/(C[1]+C[3])
    precisio.spam<-C[4]/(C[2]+C[4])
    cat('preciso email ',+precisio.email)
    cat('\nprecisio spam ',precisio.spam)
    cat('\n')
    totalprecisio.email<-totalprecisio.email + precisio.email
    totalprecisio.spam<-totalprecisio.spam + precisio.spam
}
cat('\npreciso email mitjana ',totalprecisio.email/kfold)
cat('\nprecisio spam mitjana ',totalprecisio.spam/kfold)


Matriu confusio
       Predicted
True    email spam
  email   221   39
  spam     61  139
preciso email  0.85
precisio spam  0.695

Matriu confusio
       Predicted
True    email spam
  email   244   49
  spam     53  114
preciso email  0.8327645
precisio spam  0.6826347

Matriu confusio
       Predicted
True    email spam
  email   221   69
  spam     51  119
preciso email  0.762069
precisio spam  0.7

Matriu confusio
       Predicted
True    email spam
  email   242   39
  spam     64  115
preciso email  0.86121
precisio spam  0.6424581

Matriu confusio
       Predicted
True    email spam
  email   214   56
  spam     42  148
preciso email  0.7925926
precisio spam  0.7789474

Matriu confusio
       Predicted
True    email spam
  email   242   52
  spam     63  103
preciso email  0.8231293
precisio spam  0.6204819

Matriu confusio
       Predicted
True    email spam
  email   227   47
  spam     49  137
preciso email  0.8284672
precisio spam  0.7365591

Matriu confusio
       Predict

Veiem que optenim mes precisió en $k = 10$ que en $k = 20$. Encara que la precisió de predicció de spam és més baixa que amb email.

### Regresió logística

Com que amb la regressió logística hi podem veure l'error de test no utilitzare **k-fold cross validation**.
Si no que farem un **Hold-out**, on el 10% de les dades sera de test i la restra de train.

In [23]:
x <- c(spam = 0, email = 1)
spam.categorize<-spam[-nrow(spam)]
spam.categorize$spam <- x[spam$spam]

In [24]:
ITest<-sample(1:nrow(spam),nrow(spam)/10,replace=FALSE)
spam.train<-spam[-ITest,]
spam.test<-spam[ITest,]
spam.logist<-glm(spam~.,data=spam.train,family=binomial)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

In [25]:
summary(spam.logist)
spam.pred<-predict(spam.logist,newdata=spam.test, type="response")
spam.pred.crisp<-1*(spam.pred>=0.5)
C<-table("True"=spam.test$spam,"Predicted"=spam.pred.crisp)
C


Call:
glm(formula = spam ~ ., family = binomial, data = spam.train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.0920  -0.1919   0.0000   0.1172   4.6034  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.764e+00  1.697e-01 -10.396  < 2e-16 ***
A.1         -3.368e-01  2.551e-01  -1.320 0.186741    
A.2         -1.492e-01  7.400e-02  -2.016 0.043819 *  
A.3          8.365e-02  1.162e-01   0.720 0.471637    
A.4          2.265e+00  1.508e+00   1.502 0.133041    
A.5          4.937e-01  1.025e-01   4.817 1.46e-06 ***
A.6          8.778e-01  2.626e-01   3.343 0.000830 ***
A.7          2.237e+00  3.382e-01   6.614 3.74e-11 ***
A.8          5.968e-01  1.823e-01   3.273 0.001063 ** 
A.9          1.176e+00  3.279e-01   3.586 0.000336 ***
A.10         3.678e-02  8.033e-02   0.458 0.647063    
A.11        -6.305e-01  3.224e-01  -1.956 0.050502 .  
A.12        -1.313e-01  7.692e-02  -1.707 0.087868 .  
A.13        -6.522e-02  2.412e-0

       Predicted
True      0   1
  email 275  13
  spam   26 146

In [ ]:
step(spam.logist)

Start:  AIC=1730.49
spam ~ A.1 + A.2 + A.3 + A.4 + A.5 + A.6 + A.7 + A.8 + A.9 + 
    A.10 + A.11 + A.12 + A.13 + A.14 + A.15 + A.16 + A.17 + A.18 + 
    A.19 + A.20 + A.21 + A.22 + A.23 + A.24 + A.25 + A.26 + A.27 + 
    A.28 + A.29 + A.30 + A.31 + A.32 + A.33 + A.34 + A.35 + A.36 + 
    A.37 + A.38 + A.39 + A.40 + A.41 + A.42 + A.43 + A.44 + A.45 + 
    A.46 + A.47 + A.48 + A.49 + A.50 + A.51 + A.52 + A.53 + A.54 + 
    A.55 + A.56 + A.57



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

       Df Deviance   AIC
- A.37  1     1615  1729
- A.13  1     1615  1729
- A.10  1     1615  1729
- A.31  1     1615  1729
- A.50  1     1615  1729
- A.3   1     1615  1729
- A.32  1     1615  1729
- A.18  1     1615  1729
- A.22  1     1615  1729
- A.47  1     1616  1730
- A.14  1     1616  1730
- A.1   1     1616  1730
<none>        1614  1730
- A.51  1     1617  1731
- A.30  1     1617  1731
- A.12  1     1618  1732
- A.38  1     1618  1732
- A.28  1     1618  1732
- A.11  1     1618  1732
- A.43  1     1619  1733
- A.15  1     1619  1733
- A.26  1     1620  1734
- A.34  1     1620  1734
- A.54  1     1621  1735
- A.19  1     1621  1735
- A.56  1     1621  1735
- A.39  1     1622  1736
- A.2   1     1622  1736
- A.24  1     1623  1737
- A.4   1     1623  1737
- A.55  1     1623  1737
- A.35  1     1623  1737
- A.29  1     1624  1738
- A.20  1     1625  1739
- A.33  1     1626  1740
- A.48  1     1627  1741
- A.6   1     1629  1743
- A.57  1     1629  1743
- A.49  1     1629  1743


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Step:  AIC=1728.51
spam ~ A.1 + A.2 + A.3 + A.4 + A.5 + A.6 + A.7 + A.8 + A.9 + 
    A.10 + A.11 + A.12 + A.13 + A.14 + A.15 + A.16 + A.17 + A.18 + 
    A.19 + A.20 + A.21 + A.22 + A.23 + A.24 + A.25 + A.26 + A.27 + 
    A.28 + A.29 + A.30 + A.31 + A.32 + A.33 + A.34 + A.35 + A.36 + 
    A.38 + A.39 + A.40 + A.41 + A.42 + A.43 + A.44 + A.45 + A.46 + 
    A.47 + A.48 + A.49 + A.50 + A.51 + A.52 + A.53 + A.54 + A.55 + 
    A.56 + A.57



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

       Df Deviance   AIC
- A.13  1     1615  1727
- A.10  1     1615  1727
- A.31  1     1615  1727
- A.50  1     1615  1727
- A.3   1     1615  1727
- A.32  1     1615  1727
- A.18  1     1615  1727
- A.22  1     1615  1727
- A.47  1     1616  1728
- A.14  1     1616  1728
- A.1   1     1616  1728
<none>        1615  1729
- A.51  1     1617  1729
- A.30  1     1617  1729
- A.38  1     1618  1730
- A.12  1     1618  1730
- A.28  1     1618  1730
- A.11  1     1618  1730
- A.43  1     1619  1731
- A.15  1     1619  1731
- A.26  1     1620  1732
- A.34  1     1620  1732
- A.54  1     1621  1733
- A.19  1     1621  1733
- A.56  1     1621  1733
- A.39  1     1622  1734
- A.2   1     1622  1734
- A.24  1     1623  1735
- A.4   1     1623  1735
- A.55  1     1623  1735
- A.35  1     1623  1735
- A.29  1     1624  1736
- A.20  1     1625  1737
- A.33  1     1626  1738
- A.48  1     1627  1739
- A.6   1     1629  1741
- A.57  1     1629  1741
- A.49  1     1629  1741
- A.44  1     1630  1742


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Step:  AIC=1726.58
spam ~ A.1 + A.2 + A.3 + A.4 + A.5 + A.6 + A.7 + A.8 + A.9 + 
    A.10 + A.11 + A.12 + A.14 + A.15 + A.16 + A.17 + A.18 + A.19 + 
    A.20 + A.21 + A.22 + A.23 + A.24 + A.25 + A.26 + A.27 + A.28 + 
    A.29 + A.30 + A.31 + A.32 + A.33 + A.34 + A.35 + A.36 + A.38 + 
    A.39 + A.40 + A.41 + A.42 + A.43 + A.44 + A.45 + A.46 + A.47 + 
    A.48 + A.49 + A.50 + A.51 + A.52 + A.53 + A.54 + A.55 + A.56 + 
    A.57



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

## Wine

In [18]:
nrow(wine)

[1] 178

vEiem que hi ha molt pocs necessitarem fer algun metode de resampling com boostrap


## Discriminador lineal de Fisher

In [7]:
library(MASS)

In [9]:
for (i in 1:10){
    n<- nrow(wine)
    iTrain<-sample(1:n,n,replace = TRUE)
    wine.train<-wine[iTrain,]
    wine.test<-wine[-iTrain,]
    wine.lda1<-lda(Type~.,data=wine.train)
    wine.pred<-predict(wine.lda1,newdata=wine.test,type="response")
    C<-table("True"=wine.test$Type,"Predicted"=wine.pred$class)
    cat("\n length iTest ", nrow(wine.test),'\n')
    print(C)
    cat("\n")
}


 length iTest  68 
    Predicted
True  1  2  3
   1 24  0  0
   2  1 26  0
   3  0  1 16


 length iTest  63 
    Predicted
True  1  2  3
   1 19  1  0
   2  0 26  1
   3  0  0 16


 length iTest  68 
    Predicted
True  1  2  3
   1 27  0  0
   2  1 21  0
   3  0  1 18


 length iTest  65 
    Predicted
True  1  2  3
   1 20  0  0
   2  1 25  1
   3  0  0 18


 length iTest  67 
    Predicted
True  1  2  3
   1 23  0  0
   2  0 26  0
   3  0  0 18


 length iTest  67 
    Predicted
True  1  2  3
   1 24  0  0
   2  0 25  1
   3  0  1 16


 length iTest  66 
    Predicted
True  1  2  3
   1 21  0  0
   2  0 30  0
   3  0  0 15


 length iTest  65 
    Predicted
True  1  2  3
   1 19  1  0
   2  0 24  0
   3  0  1 20


 length iTest  63 
    Predicted
True  1  2  3
   1 19  0  0
   2  2 28  1
   3  0  0 13


 length iTest  73 
    Predicted
True  1  2  3
   1 28  0  0
   2  0 25  2
   3  0  0 18



## Conctrete

In [12]:
require(regsel)
data(concrete)

conc.s<-scale(concrete)
str(conc.s)

conc.s.df<-as.data.frame(conc.s)
str(conc.s.df)

 num [1:1030, 1:9] 2.477 2.477 0.491 0.491 -0.79 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:9] "Cement" "BlastFurnaceSlag" "FlyAsh" "Water" ...
 - attr(*, "scaled:center")= Named num [1:9] 281.2 73.9 54.2 181.6 6.2 ...
  ..- attr(*, "names")= chr [1:9] "Cement" "BlastFurnaceSlag" "FlyAsh" "Water" ...
 - attr(*, "scaled:scale")= Named num [1:9] 104.51 86.28 64 21.35 5.97 ...
  ..- attr(*, "names")= chr [1:9] "Cement" "BlastFurnaceSlag" "FlyAsh" "Water" ...
'data.frame':	1030 obs. of  9 variables:
 $ Cement             : num  2.477 2.477 0.491 0.491 -0.79 ...
 $ BlastFurnaceSlag   : num  -0.856 -0.856 0.795 0.795 0.678 ...
 $ FlyAsh             : num  -0.847 -0.847 -0.847 -0.847 -0.847 ...
 $ Water              : num  -0.916 -0.916 2.174 2.174 0.489 ...
 $ Superplasticizer   : num  -0.62 -0.62 -1.04 -1.04 -1.04 ...
 $ CoarseAggregate    : num  0.8627 1.0557 -0.5263 -0.5263 0.0705 ...
 $ FineAggregate      : num  -1.217 -1.217 -2.24 -2.24 0.648 ...
 $ Age            

In [14]:
conc.ls1<-lm(Cement~BlastFurnaceSlag+FlyAsh+Water+Superplasticizer+CoarseAggregate+FineAggregate+Age+CompressiveStrength, data=conc.s.df)
summary(conc.ls1)


Call:
lm(formula = Cement ~ BlastFurnaceSlag + FlyAsh + Water + Superplasticizer + 
    CoarseAggregate + FineAggregate + Age + CompressiveStrength, 
    data = conc.s.df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.11042 -0.22021 -0.03327  0.18994  1.76766 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)         -6.559e-16  1.046e-02   0.000    1.000    
BlastFurnaceSlag    -8.505e-01  1.303e-02 -65.278   <2e-16 ***
FlyAsh              -7.413e-01  1.301e-02 -56.967   <2e-16 ***
Water               -5.279e-01  2.245e-02 -23.515   <2e-16 ***
Superplasticizer    -2.778e-02  1.807e-02  -1.537    0.125    
CoarseAggregate     -5.540e-01  1.602e-02 -34.578   <2e-16 ***
FineAggregate       -7.155e-01  1.637e-02 -43.724   <2e-16 ***
Age                 -1.139e-01  1.276e-02  -8.929   <2e-16 ***
CompressiveStrength  2.178e-01  1.543e-02  14.113   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual s

In [15]:
step(conc.ls1)

Start:  AIC=-2240.39
Cement ~ BlastFurnaceSlag + FlyAsh + Water + Superplasticizer + 
    CoarseAggregate + FineAggregate + Age + CompressiveStrength

                      Df Sum of Sq    RSS      AIC
<none>                             114.97 -2240.39
- Superplasticizer     1      0.27 115.24 -2240.01
- Age                  1      8.98 123.95 -2164.95
- CompressiveStrength  1     22.43 137.40 -2058.83
- Water                1     62.27 177.24 -1796.61
- CoarseAggregate      1    134.64 249.62 -1443.91
- FineAggregate        1    215.28 330.25 -1155.58
- FlyAsh               1    365.44 480.42  -769.54
- BlastFurnaceSlag     1    479.85 594.82  -549.52



Call:
lm(formula = Cement ~ BlastFurnaceSlag + FlyAsh + Water + Superplasticizer + 
    CoarseAggregate + FineAggregate + Age + CompressiveStrength, 
    data = conc.s.df)

Coefficients:
        (Intercept)     BlastFurnaceSlag               FlyAsh  
         -6.559e-16           -8.505e-01           -7.413e-01  
              Water     Superplasticizer      CoarseAggregate  
         -5.279e-01           -2.778e-02           -5.540e-01  
      FineAggregate                  Age  CompressiveStrength  
         -7.155e-01           -1.139e-01            2.178e-01  
